## tradeSeq for DE along the Mullerian duct developmental trajectory

In [ ]:
#if (!require("BiocManager", quietly = TRUE))
#    install.packages("BiocManager")

#BiocManager::install("tradeSeq")

In [ ]:
library(tradeSeq)
library(SingleCellExperiment)
library(slingshot)

In [ ]:
adata <- readRDS('/nfs/team292/vl6/FetalReproductiveTract/mullerian_early.rds')

In [ ]:
adata

In [ ]:
adata <- slingshot(adata, clusterLabels = 'leiden', 
                   start.clus = "6", end.clus = c("1", "7", "8"), reducedDim = 'X_draw_graph_fa', 
                  )

In [ ]:
counts <- assay(adata, "X")


In [ ]:
adata


In [ ]:
head(colData(adata)$slingPseudotime_1)

In [ ]:
library(grDevices)
library(RColorBrewer)
colors <- colorRampPalette(brewer.pal(11,'Spectral')[-6])(100)
plotcol <- colors[cut(adata$slingPseudotime_1, breaks=100)]

plot(reducedDims(adata)$X_draw_graph_fa, col = plotcol, pch=16, asp = 1)
lines(SlingshotDataSet(adata), type = 'lineages' , lwd=2, col='black')

In [ ]:
library(grDevices)
library(RColorBrewer)
colors <- colorRampPalette(brewer.pal(11,'Spectral')[-6])(100)
plotcol <- colors[cut(adata$slingPseudotime_2, breaks=100)]

plot(reducedDims(adata)$X_draw_graph_fa, col = plotcol, pch=16, asp = 1)
lines(SlingshotDataSet(adata), type = 'lineages' , lwd=2, col='black')

In [ ]:
library(grDevices)
library(RColorBrewer)
colors <- colorRampPalette(brewer.pal(11,'Spectral')[-6])(100)
plotcol <- colors[cut(adata$slingPseudotime_3, breaks=100)]

plot(reducedDims(adata)$X_draw_graph_fa, col = plotcol, pch=16, asp = 1)
lines(SlingshotDataSet(adata), type = 'lineages' , lwd=2, col='black')

In [ ]:
adata

In [ ]:
slotNames(SlingshotDataSet(adata))

In [ ]:
SlingshotDataSet(adata)@lineages

In [ ]:
 brewer.pal(9,'Set1')

In [ ]:
sort(unique(adata$leiden))

In [ ]:
brewer.pal(9,'Set1')

In [ ]:
palette = c('#EEE8AA', '#FFD700', '#ADFF2F', '#66CDAA', '#FFFF00', '#90EE90', '#BDB76B',
            '#008080', '#9ACD32', '#FFE4B5', '#db8b96')

In [ ]:
pdf('/home/jovyan/RepTract/FA_lineages.pdf', width = 6, height = 6.5)
plot(reducedDims(adata)$X_draw_graph_fa, col = palette[adata$leiden], pch=16, asp = 0.4)
lines(SlingshotDataSet(adata), type = 'lineages' , lwd=1.5, col='black')
legend("topright", legend=sort(unique(adata$leiden)), title="leiden", 
       col = palette, pch = 16) 
dev.off()

In [ ]:
palette_stage = c('#440154', '#482878', '#3e4989', '#31688e', '#26828e', '#1f9e89', '#35b779', '#6ece58', '#b5de2b', '#fde725')

In [ ]:
pdf('/home/jovyan/RepTract/FA_stages.pdf', width = 6, height = 6)
plot(reducedDims(adata)$X_draw_graph_fa, col = adata$stage_pcw, palette = palette_stage, pch=16, asp = 0.7)
#lines(SlingshotDataSet(adata), type = 'lineages' , lwd=2, col='black')
#legend("topright", legend=sort(unique(adata$stage)), title="stage", 
#       col = palette, pch = 16) 
dev.off()

In [ ]:
palette_sex = c('#FFB6C1', '#AFEEEE')

In [ ]:
adata$sex <- factor(adata$sex, levels = c("female", "male"))

In [ ]:
pdf('/home/jovyan/RepTract/FA_sex.pdf', width = 6, height = 6.5)
palette_sex_transparent <- sapply(palette_sex, adjustcolor)
plot(reducedDims(adata)$X_draw_graph_fa, col = palette_sex_transparent[adata$sex], pch=16, asp = 0.4)
#lines(SlingshotDataSet(adata), type = 'lineages' , lwd=2, col='black')
legend("topright", legend=sort(unique(adata$sex)), title="sex", 
       col = palette_sex, pch = 16) 
dev.off()

In [ ]:
adata

In [ ]:
SlingshotDataSet(adata)

In [ ]:
set.seed(7)
pseudotime <- slingPseudotime(SlingshotDataSet(adata), na = FALSE)
cellWeights <- slingCurveWeights(SlingshotDataSet(adata))

In [ ]:
library(scran)

In [ ]:
libsizes <- colSums(counts)
size.factors <- libsizes/mean(libsizes)
logcounts(adata) <- log2(t(t(counts)/size.factors) + 1)
assayNames(adata)

In [ ]:
# Filter genes with non-zero biological components
getVariableGenes <- function(adata,fdr_thresh=NULL,var_thresh=0) {
  gene_var <- modelGeneVar(adata)
  genes <- getTopHVGs(gene_var, var.threshold =var_thresh ,fdr.threshold = fdr_thresh)
  return(genes)
}

In [ ]:
genes <- getVariableGenes(adata)

In [ ]:
length(genes)

In [ ]:
dim(counts[genes,])

In [ ]:
# icMat <- evaluateK(counts = counts, sds = SlingshotDataSet(adata), k = 5:15, 
#                    nGenes = 200, verbose = T)

In [ ]:
sce <- fitGAM(counts = counts[genes,], pseudotime = pseudotime, cellWeights = cellWeights, nknots = 8,
                verbose = TRUE, parallel = TRUE)

In [ ]:
table(rowData(sce)$tradeSeq$converged)

In [ ]:
adata <- adata[genes, ]

In [ ]:
adata

In [ ]:
saveRDS(sce, "/nfs/team292/vl6/FetalReproductiveTract/mullerian_emergence_gam.rds")

In [ ]:
sce <- readRDS('/nfs/team292/vl6/FetalReproductiveTract/mullerian_emergence_gam.rds')

In [ ]:
sce

In [ ]:
assoRes <- associationTest(sce, lineages = TRUE)
head(assoRes)

In [ ]:
write.csv(assoRes, file = "/home/jovyan/SupplementaryTable6.csv")


In [ ]:
dim(assoRes)

In [ ]:
assoRes['PNOC',]

In [ ]:
assoRes['TMEM163',]

In [ ]:
assoRes['SP7',]

In [ ]:
assoRes['MMP28',]

In [ ]:
assoRes['RXRG',]

In [ ]:
assoRes['FGF20',]

In [ ]:
assoRes['LYPD1',]

In [ ]:
assoRes['WIF1',]

In [ ]:
assoRes['GDNF',]

In [ ]:
assoRes['CALCA',]

In [ ]:
assoRes['NOTUM',]

In [ ]:
assoRes['NKD1',]

In [ ]:
assoRes['GAL',]

## Filtering across lineages

In [ ]:
assoRes = assoRes[assoRes$pvalue < 0.001,]
dim(assoRes)

In [ ]:
assoRes = assoRes[assoRes$waldStat > 100,]
dim(assoRes)

## Filtering within lineages

In [ ]:
library(dplyr)

In [ ]:
lineage1 = assoRes %>% filter(waldStat_1 >= 150) %>% filter(pvalue_1 < 0.0001) %>% filter(waldStat_3 < 300) %>% filter(waldStat_2 < 300)

In [ ]:
dim(lineage1)

In [ ]:
write.csv(lineage1, "/home/jovyan/MüllerianEmergence_TradeSeq_MeseMale.csv", row.names = TRUE)

In [ ]:
lineage2 = assoRes %>% filter(waldStat_2 >= 150) %>% filter(pvalue_2 < 0.0001) %>% filter(waldStat_3 < 150) %>% filter(waldStat_1 < 150)

In [ ]:
dim(lineage2)

In [ ]:
write.csv(lineage2, "/home/jovyan/MüllerianEmergence_TradeSeq_Epi.csv", row.names = TRUE)

In [ ]:
lineage3 = assoRes %>% filter(waldStat_3 >= 150) %>% filter(pvalue_3 < 0.0001) %>% filter(waldStat_2 < 300) %>% filter(waldStat_1 < 300)

In [ ]:
dim(lineage3)

In [ ]:
write.csv(lineage3, "/home/jovyan/MüllerianEmergence_TradeSeq_MeseFemale.csv", row.names = TRUE)

In [ ]:
lineage1['ADRA2A',]

# Plot selected markers per lineage

## Mullerian mesenchyme female

In [ ]:
# PELI1 
PELI1 <- plotSmoothers(sce, counts, gene = 'PELI1', alpha =0, 
                     xlab = "pseudotime",
  ylab = "Log(expression + 1)", border = FALSE) + ggtitle("PELI1") + theme(title = element_text(size = 20, face = 'bold'), axis.title=element_text(size=20), axis.text=element_text(size=20), legend.text = element_text(size=20),
     legend.title = element_text(size=20, face = "bold"))+ theme_light()+ ylim(0, 2)
PELI1

In [ ]:
# TMEM163 
TMEM163 <- plotSmoothers(sce, counts, gene = 'TMEM163', alpha =0, 
                     xlab = "pseudotime",
  ylab = "Log(expression + 1)", border = FALSE) + ggtitle("TMEM163") + theme(title = element_text(size = 20, face = 'bold'), axis.title=element_text(size=20), axis.text=element_text(size=20), legend.text = element_text(size=20),
     legend.title = element_text(size=20, face = "bold"))+ theme_light()+ ylim(0, 1.4)
TMEM163

In [ ]:
# EVA1B 
EVA1B <- plotSmoothers(sce, counts, gene = 'ALDH1A1', alpha =0, 
                     xlab = "pseudotime",
  ylab = "Log(expression + 1)", border = FALSE) + ggtitle("ALDH1A1") + theme(title = element_text(size = 20, face = 'bold'), axis.title=element_text(size=20), axis.text=element_text(size=20), legend.text = element_text(size=20),
     legend.title = element_text(size=20, face = "bold"))+ theme_light()+ ylim(0, 3)
EVA1B

In [ ]:
# PLXDC1 
PLXDC1 <- plotSmoothers(sce, counts, gene = 'PLXDC1', alpha =0, 
                     xlab = "pseudotime",
  ylab = "Log(expression + 1)", border = FALSE) + ggtitle("PLXDC1") + theme(title = element_text(size = 20, face = 'bold'), axis.title=element_text(size=20), axis.text=element_text(size=20), legend.text = element_text(size=20),
     legend.title = element_text(size=20, face = "bold"))+ theme_light()+ ylim(0, 1.5)
PLXDC1

In [ ]:
# ZAP70 
ZAP70 <- plotSmoothers(sce, counts, gene = 'ZAP70', alpha =0, 
                     xlab = "pseudotime",
  ylab = "Log(expression + 1)", border = FALSE) + ggtitle("ZAP70") + theme(title = element_text(size = 20, face = 'bold'), axis.title=element_text(size=20), axis.text=element_text(size=20), legend.text = element_text(size=20),
     legend.title = element_text(size=20, face = "bold"))+ theme_light()+ ylim(0, 1)
ZAP70

In [ ]:
# EDNRA 
EDNRA <- plotSmoothers(sce, counts, gene = 'EDNRA', alpha =0, 
                     xlab = "pseudotime",
  ylab = "Log(expression + 1)", border = FALSE) + ggtitle("EDNRA") + theme(title = element_text(size = 20, face = 'bold'), axis.title=element_text(size=20), axis.text=element_text(size=20), legend.text = element_text(size=20),
     legend.title = element_text(size=20, face = "bold"))+ theme_light()+ ylim(0, 3)
EDNRA

In [ ]:
# EDNRA 
ENTPD1 <- plotSmoothers(sce, counts, gene = 'ENTPD1', alpha =0, 
                     xlab = "pseudotime",
  ylab = "Log(expression + 1)", border = FALSE) + ggtitle("ENTPD1") + theme(title = element_text(size = 20, face = 'bold'), axis.title=element_text(size=20), axis.text=element_text(size=20), legend.text = element_text(size=20),
     legend.title = element_text(size=20, face = "bold"))+ theme_light()+ ylim(0, 1)
ENTPD1


In [ ]:
# MMP28 
THBS4 <- plotSmoothers(sce, counts, gene = 'THBS4', alpha =0, 
                     xlab = "pseudotime",
  ylab = "Log(expression + 1)", border = FALSE) + ggtitle("THBS4") + theme(title = element_text(size = 20, face = 'bold'), axis.title=element_text(size=20), axis.text=element_text(size=20), legend.text = element_text(size=20),
     legend.title = element_text(size=20, face = "bold"))+ theme_light()+ ylim(0, 1.8)
THBS4

In [ ]:
# ECM1 
ECM1 <- plotSmoothers(sce, counts, gene = 'ECM1', alpha =0, 
                     xlab = "pseudotime",
  ylab = "Log(expression + 1)", border = FALSE) + ggtitle("ECM1") + theme(title = element_text(size = 20, face = 'bold'), axis.title=element_text(size=20), axis.text=element_text(size=20), legend.text = element_text(size=20),
     legend.title = element_text(size=20, face = "bold"))+ theme_light()+ ylim(0, 1)
ECM1

In [ ]:
# TNFRSF21 
TNFRSF21 <- plotSmoothers(sce, counts, gene = 'TNFRSF21', alpha =0, 
                     xlab = "pseudotime",
  ylab = "Log(expression + 1)", border = FALSE) + ggtitle("TNFRSF21") + theme(title = element_text(size = 20, face = 'bold'), axis.title=element_text(size=20), axis.text=element_text(size=20), legend.text = element_text(size=20),
     legend.title = element_text(size=20, face = "bold"))+ theme_light()+ ylim(0, 1.3)
TNFRSF21


In [ ]:
# CXCR4 
CXCR4 <- plotSmoothers(sce, counts, gene = 'CXCR4', alpha =0, 
                     xlab = "pseudotime",
  ylab = "Log(expression + 1)", border = FALSE) + ggtitle("CXCR4") + theme(title = element_text(size = 20, face = 'bold'), axis.title=element_text(size=20), axis.text=element_text(size=20), legend.text = element_text(size=20),
     legend.title = element_text(size=20, face = "bold"))+ theme_light()+ ylim(0, 1.4)
CXCR4


In [ ]:
# LAMC3 
LAMC3 <- plotSmoothers(sce, counts, gene = 'LAMC3', alpha =0, 
                     xlab = "pseudotime",
  ylab = "Log(expression + 1)", border = FALSE) + ggtitle("LAMC3") + theme(title = element_text(size = 20, face = 'bold'), axis.title=element_text(size=20), axis.text=element_text(size=20), legend.text = element_text(size=20),
     legend.title = element_text(size=20, face = "bold"))+ theme_light()+ ylim(0, 1)
LAMC3


In [ ]:
# IGFBP3 
IGFBP3 <- plotSmoothers(sce, counts, gene = 'IGFBP3', alpha =0, 
                     xlab = "pseudotime",
  ylab = "Log(expression + 1)", border = FALSE) + ggtitle("IGFBP3") + theme(title = element_text(size = 20, face = 'bold'), axis.title=element_text(size=20), axis.text=element_text(size=20), legend.text = element_text(size=20),
     legend.title = element_text(size=20, face = "bold"))+ theme_light()+ ylim(0, 3)
IGFBP3


In [ ]:
# PCLAF 
PCLAF <- plotSmoothers(sce, counts, gene = 'PCLAF', alpha =0, 
                     xlab = "pseudotime",
  ylab = "Log(expression + 1)", border = FALSE) + ggtitle("PCLAF") + theme(title = element_text(size = 20, face = 'bold'), axis.title=element_text(size=20), axis.text=element_text(size=20), legend.text = element_text(size=20),
     legend.title = element_text(size=20, face = "bold"))+ theme_light()+ ylim(0, 2)
PCLAF


In [ ]:
# PCLAF 
CNTN1 <- plotSmoothers(sce, counts, gene = 'CNTN1', alpha =0, 
                     xlab = "pseudotime",
  ylab = "Log(expression + 1)", border = FALSE) + ggtitle("CNTN1") + theme(title = element_text(size = 20, face = 'bold'), axis.title=element_text(size=20), axis.text=element_text(size=20), legend.text = element_text(size=20),
     legend.title = element_text(size=20, face = "bold"))+ theme_light()+ ylim(0, 1.7)
CNTN1


In [ ]:
# PCLAF 
NID1 <- plotSmoothers(sce, counts, gene = 'NID1', alpha =0, 
                     xlab = "pseudotime",
  ylab = "Log(expression + 1)", border = FALSE) + ggtitle("NID1") + theme(title = element_text(size = 20, face = 'bold'), axis.title=element_text(size=20), axis.text=element_text(size=20), legend.text = element_text(size=20),
     legend.title = element_text(size=20, face = "bold"))+ theme_light()+ ylim(0, 1.2)
NID1


In [ ]:
# PCLAF 
CPED1 <- plotSmoothers(sce, counts, gene = 'CPED1', alpha =0, 
                     xlab = "pseudotime",
  ylab = "Log(expression + 1)", border = FALSE) + ggtitle("CPED1") + theme(title = element_text(size = 20, face = 'bold'), axis.title=element_text(size=20), axis.text=element_text(size=20), legend.text = element_text(size=20),
     legend.title = element_text(size=20, face = "bold"))+ theme_light()+ ylim(0, 1.3)
CPED1

### Save plots

In [ ]:
library(gridExtra)

In [ ]:
p_mullerian_mese_female <- grid.arrange(CNTN1, PLXDC1, TMEM163, ZAP70, PELI1, THBS4,
                                        nrow = 1, ncol = 6)

In [ ]:
ggsave('/home/jovyan/RepTract/figures-epithelial/mullerian_mesenchyme_female_tradeseq.pdf',
       plot = p_mullerian_mese_female,
       width = 21, height = 3, dpi = 300)

## Mullerian mesenchyme male

In [ ]:
# HRC 
HRC <- plotSmoothers(sce, counts, gene = 'HRC', alpha =0, 
                     xlab = "pseudotime",
  ylab = "Log(expression + 1)", border = FALSE) + ggtitle("HRC") + theme(title = element_text(size = 20, face = 'bold'), axis.title=element_text(size=20), axis.text=element_text(size=20), legend.text = element_text(size=20),
     legend.title = element_text(size=20, face = "bold"))+ theme_light()+ ylim(0, 1)
HRC


In [ ]:
# HRC 
LAMP5 <- plotSmoothers(sce, counts, gene = 'LAMP5', alpha =0, 
                     xlab = "pseudotime",
  ylab = "Log(expression + 1)", border = FALSE) + ggtitle("LAMP5") + theme(title = element_text(size = 20, face = 'bold'), axis.title=element_text(size=20), axis.text=element_text(size=20), legend.text = element_text(size=20),
     legend.title = element_text(size=20, face = "bold"))+ theme_light()+ ylim(0, 1.2)
LAMP5


In [ ]:
# HRC 
DLX5 <- plotSmoothers(sce, counts, gene = 'DLX5', alpha =0, 
                     xlab = "pseudotime",
  ylab = "Log(expression + 1)", border = FALSE) + ggtitle("DLX5") + theme(title = element_text(size = 20, face = 'bold'), axis.title=element_text(size=20), axis.text=element_text(size=20), legend.text = element_text(size=20),
     legend.title = element_text(size=20, face = "bold"))+ theme_light()+ ylim(0, 1.3)
DLX5


In [ ]:
# NOTUM 
NOTUM <- plotSmoothers(sce, counts, gene = 'NOTUM', alpha =0, 
                     xlab = "pseudotime",
  ylab = "Log(expression + 1)", border = FALSE) + ggtitle("NOTUM") + theme(title = element_text(size = 20, face = 'bold'), axis.title=element_text(size=20), axis.text=element_text(size=20), legend.text = element_text(size=20),
     legend.title = element_text(size=20, face = "bold"))+ theme_light()+ ylim(0, 1.7)
NOTUM


In [ ]:
# WIF1 
WIF1 <- plotSmoothers(sce, counts, gene = 'WIF1', alpha =0, 
                     xlab = "pseudotime",
  ylab = "Log(expression + 1)", border = FALSE) + ggtitle("WIF1") + theme(title = element_text(size = 20, face = 'bold'), axis.title=element_text(size=20), axis.text=element_text(size=20), legend.text = element_text(size=20),
     legend.title = element_text(size=20, face = "bold"))+ theme_light()+ ylim(0, 1.3)
WIF1


In [ ]:
# SP7 
SP7 <- plotSmoothers(sce, counts, gene = 'SP7', alpha =0, 
                     xlab = "pseudotime",
  ylab = "Log(expression + 1)", border = FALSE) + ggtitle("SP7") + theme(title = element_text(size = 20, face = 'bold'), axis.title=element_text(size=20), axis.text=element_text(size=20), legend.text = element_text(size=20),
     legend.title = element_text(size=20, face = "bold"))+ theme_light()+ ylim(0, 1.2)
SP7


In [ ]:
# NKD1 
NKD1 <- plotSmoothers(sce, counts, gene = 'NKD1', alpha =0, 
                     xlab = "pseudotime",
  ylab = "Log(expression + 1)", border = FALSE) + ggtitle("NKD1") + theme(title = element_text(size = 20, face = 'bold'), axis.title=element_text(size=20), axis.text=element_text(size=20), legend.text = element_text(size=20),
     legend.title = element_text(size=20, face = "bold"))+ theme_light()+ ylim(0, 2.3)
NKD1


In [ ]:
# AR 
AR <- plotSmoothers(sce, counts, gene = 'AR', alpha =0, 
                     xlab = "pseudotime",
  ylab = "Log(expression + 1)", border = FALSE) + ggtitle("AR") + theme(title = element_text(size = 20, face = 'bold'), axis.title=element_text(size=20), axis.text=element_text(size=20), legend.text = element_text(size=20),
     legend.title = element_text(size=20, face = "bold"))+ theme_light()+ ylim(0, 0.75)
AR


In [ ]:
# AR 
BCAN <- plotSmoothers(sce, counts, gene = 'BCAN', alpha =0, 
                     xlab = "pseudotime",
  ylab = "Log(expression + 1)", border = FALSE) + ggtitle("BCAN") + theme(title = element_text(size = 20, face = 'bold'), axis.title=element_text(size=20), axis.text=element_text(size=20), legend.text = element_text(size=20),
     legend.title = element_text(size=20, face = "bold"))+ theme_light()+ ylim(0, 1)
BCAN


In [ ]:
# AR 
GAL <- plotSmoothers(sce, counts, gene = 'GAL', alpha =0, 
                     xlab = "pseudotime",
  ylab = "Log(expression + 1)", border = FALSE) + ggtitle("GAL") + theme(title = element_text(size = 20, face = 'bold'), axis.title=element_text(size=20), axis.text=element_text(size=20), legend.text = element_text(size=20),
     legend.title = element_text(size=20, face = "bold"))+ theme_light()+ ylim(0, 0.65)
GAL


In [ ]:
# AR 
COX7A1 <- plotSmoothers(sce, counts, gene = 'COX7A1', alpha =0, 
                     xlab = "pseudotime",
  ylab = "Log(expression + 1)", border = FALSE) + ggtitle("COX7A1") + theme(title = element_text(size = 20, face = 'bold'), axis.title=element_text(size=20), axis.text=element_text(size=20), legend.text = element_text(size=20),
     legend.title = element_text(size=20, face = "bold"))+ theme_light()+ ylim(0, 0.75)
COX7A1

In [ ]:
# AR 
ADRA2A <- plotSmoothers(sce, counts, gene = 'ADRA2A', alpha =0, 
                     xlab = "pseudotime",
  ylab = "Log(expression + 1)", border = FALSE) + ggtitle("ADRA2A") + theme(title = element_text(size = 20, face = 'bold'), axis.title=element_text(size=20), axis.text=element_text(size=20), legend.text = element_text(size=20),
     legend.title = element_text(size=20, face = "bold"))+ theme_light()+ ylim(0, 0.9)
ADRA2A

In [ ]:
p_mullerian_mese_male <- grid.arrange(WIF1, SP7, NKD1, NOTUM, ADRA2A, GAL,  nrow = 1, ncol = 6)

In [ ]:
ggsave('/home/jovyan/RepTract/figures-epithelial/mullerian_mesenchyme_male_tradeseq.pdf',
       plot = p_mullerian_mese_male,
       width = 21, height = 3, dpi = 300)


## Mullerian epithelium

In [ ]:
head(endRes_sig)

In [ ]:
lineage2 <- endRes_sig %>% filter(logFC1_2 < -4.5 & logFC2_3 > 4.5) %>% filter(waldStat > 70) %>%
arrange(desc(waldStat)) 

In [ ]:
dim(lineage2)

In [ ]:
endRes_sig['FGF20', ]

In [ ]:
lineage2_top <- row.names(lineage2)[1:69]

In [ ]:
lineage2_top

In [ ]:
# Write the character vector to a text file
writeLines(lineage2_top, "/nfs/team292/vl6/FetalReproductiveTract/mullerian_epi_genes.txt")

In [ ]:
for (gene in lineage2_top) {
    print(gene)
    print(plotSmoothers(sce, counts, gene = gene, alpha = 0) + ggtitle(gene) +
                          theme(title = element_text(size = 20, face = 'bold'),
                                axis.title = element_text(size = 20),
                                axis.text = element_text(size = 20),
                                legend.text = element_text(size = 20),
                                legend.title = element_text(size = 20, face = "bold")))
}

In [ ]:
# FGF20 
FGF20 <- plotSmoothers(sce, counts, gene = 'FGF20', alpha =0, 
                     xlab = "pseudotime",
  ylab = "Log(expression + 1)", border = FALSE) + ggtitle("FGF20") + theme(title = element_text(size = 20, face = 'bold'), axis.title=element_text(size=20), axis.text=element_text(size=20), legend.text = element_text(size=20),
     legend.title = element_text(size=20, face = "bold"))+ theme_light()+ ylim(0, 1.5)
FGF20


In [ ]:
# RXRG 
RXRG <- plotSmoothers(sce, counts, gene = 'RXRG', alpha =0, 
                     xlab = "pseudotime",
  ylab = "Log(expression + 1)", border = FALSE) + ggtitle("RXRG") + theme(title = element_text(size = 20, face = 'bold'), axis.title=element_text(size=20), axis.text=element_text(size=20), legend.text = element_text(size=20),
     legend.title = element_text(size=20, face = "bold"))+ theme_light()+ ylim(0, 1)
RXRG


In [ ]:
# RXRG 
PNOC <- plotSmoothers(sce, counts, gene = 'PNOC', alpha =0, 
                     xlab = "pseudotime",
  ylab = "Log(expression + 1)", border = FALSE) + ggtitle("PNOC") + theme(title = element_text(size = 20, face = 'bold'), axis.title=element_text(size=20), axis.text=element_text(size=20), legend.text = element_text(size=20),
     legend.title = element_text(size=20, face = "bold"))+ theme_light()+ ylim(0, 3)
PNOC

In [ ]:
# RXRG 
LYPD1 <- plotSmoothers(sce, counts, gene = 'LYPD1', alpha =0, 
                     xlab = "pseudotime",
  ylab = "Log(expression + 1)", border = FALSE) + ggtitle("LYPD1") + theme(title = element_text(size = 20, face = 'bold'), axis.title=element_text(size=20), axis.text=element_text(size=20), legend.text = element_text(size=20),
     legend.title = element_text(size=20, face = "bold"))+ theme_light()+ ylim(0, 3)
LYPD1

In [ ]:
# SSTR2 
SSTR2 <- plotSmoothers(sce, counts, gene = 'SSTR2', alpha =0, 
                     xlab = "pseudotime",
  ylab = "Log(expression + 1)", border = FALSE) + ggtitle("SSTR2") + theme(title = element_text(size = 20, face = 'bold'), axis.title=element_text(size=20), axis.text=element_text(size=20), legend.text = element_text(size=20),
     legend.title = element_text(size=20, face = "bold"))+ theme_light()+ ylim(0, 1.5)
SSTR2


In [ ]:
# SSTR2 
CHL1 <- plotSmoothers(sce, counts, gene = 'CHL1', alpha =0, 
                     xlab = "pseudotime",
  ylab = "Log(expression + 1)", border = FALSE) + ggtitle("CHL1") + theme(title = element_text(size = 20, face = 'bold'), axis.title=element_text(size=20), axis.text=element_text(size=20), legend.text = element_text(size=20),
     legend.title = element_text(size=20, face = "bold"))+ theme_light()+ ylim(0, 2)
CHL1

In [ ]:
# SSTR2 
LGI1 <- plotSmoothers(sce, counts, gene = 'LGI1', alpha =0, 
                     xlab = "pseudotime",
  ylab = "Log(expression + 1)", border = FALSE) + ggtitle("LGI1") + theme(title = element_text(size = 20, face = 'bold'), axis.title=element_text(size=20), axis.text=element_text(size=20), legend.text = element_text(size=20),
     legend.title = element_text(size=20, face = "bold"))+ theme_light()+ ylim(0, 1.3)
LGI1

In [ ]:
# SSTR2 
CALCA <- plotSmoothers(sce, counts, gene = 'CALCA', alpha =0, 
                     xlab = "pseudotime",
  ylab = "Log(expression + 1)", border = FALSE) + ggtitle("CALCA") + theme(title = element_text(size = 20, face = 'bold'), axis.title=element_text(size=20), axis.text=element_text(size=20), legend.text = element_text(size=20),
     legend.title = element_text(size=20, face = "bold"))+ theme_light()+ ylim(0, 0.75)
CALCA

In [ ]:
# GDNF 
GDNF <- plotSmoothers(sce, counts, gene = 'GDNF', alpha =0, 
                     xlab = "pseudotime",
  ylab = "Log(expression + 1)", border = FALSE) + ggtitle("GDNF") + theme(title = element_text(size = 20, face = 'bold'), axis.title=element_text(size=20), axis.text=element_text(size=20), legend.text = element_text(size=20),
     legend.title = element_text(size=20, face = "bold"))+ theme_light()+ ylim(0, 0.7)
GDNF

In [ ]:
p_mullerian_epi <- grid.arrange(RXRG, PNOC, LYPD1, FGF20, SSTR2, GDNF, nrow = 1, ncol = 6)

In [ ]:
ggsave('/home/jovyan/RepTract/figures-epithelial/mullerian_epithelium_tradeseq.pdf',
       plot = p_mullerian_epi,
       width = 21, height = 3, dpi = 300)


In [ ]:
p_mullerian_supp <- grid.arrange(CALCA, SSTR2, LGI1, LAMP5, nrow = 1, ncol = 4)

In [ ]:
ggsave('/home/jovyan/RepTract/figures-epithelial/mullerian_supplementary_tradeseq.pdf',
       plot = p_mullerian_supp,
       width = 14, height = 3, dpi = 300)
